# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
clients = pd.read_csv('/datasets/data.csv')
print(clients.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [2]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Рассмотрим полученную информацию.

Всего в таблице 12 столбцов и 21525 записей (строк)

children — количество детей в семье;
days_employed — общий трудовой стаж в днях;
dob_years — возраст клиента в годах;
education — уровень образования клиента;
education_id — идентификатор уровня образования;
family_status — семейное положение;
family_status_id — идентификатор семейного положения;
gender — пол клиента;
income_type — тип занятости;
debt — имел ли задолженность по возврату кредитов;
total_income — ежемесячный доход;
purpose — цель получения кредита.

Каждая строка таблицы содержит общую информацию о клиентах банка, уникальных user_id в данных не предусмотрено. Наименование столбцов в датафрейме указано корректно. Однако количество значений в столбцах различается, а это означает, что в данных есть пропущенные значения. Таким образом, главная задача, которую необходимо решить на этапе предобработки данных - ликвидация пропущенных значений и подготовка данный для дальнейшей работы. 

Для выполнения требований Заказчика необходимо уделить особое внимание стобцам children, family_status, debt, total_income, purpose.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# Проверим данные на наличие пропусков:
clients.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах пропущено 2174 значения, что составляет 10% всей выборки. Это довольно высокое значение, значит нужно разбираться с пропущенными значениями более детально.
Трудовой стаж в днях (days_employed) и ежемесячный доход (total_income) - количественные переменные. Информация по столбцу с ежемесячным доходом характеризует состояние выборки и влияет на итоговый результат. Для примерной оценки значений выборки по столбцу total_income используем медиану с учетом типа занятости клиента.

In [4]:
# Посмотрим, какие уникальные значения есть в столбце income_type
print(clients['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Рассчитаем медиану ежемесячного дохода по каждому типу занятости

In [5]:
employee_data = clients[clients['income_type'] == 'сотрудник']
#print(employee_data.head(10))
employee_data_income_median = employee_data['total_income'].median()
print('Медиана ежемесячного дохода для сотрудника составляет:', employee_data_income_median)

companion_data = clients[clients['income_type'] == 'компаньон']
#print(companion_data.head(10))
companion_data_income_median = companion_data['total_income'].median()
print('Медиана ежемесячного дохода для компаньона составляет:', companion_data_income_median)

retiree_data = clients[clients['income_type'] == 'пенсионер']
#print(retiree_data.head(10))
retiree_data_income_median = retiree_data['total_income'].median()
print('Медиана ежемесячного дохода для пенсионера составляет:', retiree_data_income_median)

civil_servant_data = clients[clients['income_type'] == 'госслужащий']
#print(civil_servant_data.head(10))
civil_servant_data_income_median = civil_servant_data['total_income'].median()
print('Медиана ежемесячного дохода для госслужащего составляет:', civil_servant_data_income_median)

unemployed_data = clients[clients['income_type'] == 'безработный']
#print(unemployed_data.head(5))

entrepreneur_data = clients[clients['income_type'] == 'предприниматель']
#print(entrepreneur_data.head(5))
entrepreneur_data_income_median = entrepreneur_data['total_income'].median()
print('Медиана ежемесячного дохода для предпринимателя составляет:', entrepreneur_data_income_median)

student_data = clients[clients['income_type'] == 'студент']
#print(student_data.head(5))

decree_data = clients[clients['income_type'] == 'в декрете']
#print(decree_data.head(5))

Медиана ежемесячного дохода для сотрудника составляет: 142594.39684740017
Медиана ежемесячного дохода для компаньона составляет: 172357.95096577113
Медиана ежемесячного дохода для пенсионера составляет: 118514.48641164352
Медиана ежемесячного дохода для госслужащего составляет: 150447.9352830068
Медиана ежемесячного дохода для предпринимателя составляет: 499163.1449470857


Заменим пропущенные значения в столбце total_income с учетом рассчитанных медиан ежемесячного дохода для следующих типов занятости: 'сотрудник', 'компаньон', пенсионер', 'госслужащий'.
Для типов занятости 'безработный', 'студент', 'в декрете' пропущенные значения отсутствуют.

In [6]:
clients.loc[(clients['total_income'].isnull()) & (clients['income_type'] == 'сотрудник'), 'total_income'] = employee_data_income_median
clients.loc[(clients['total_income'].isnull()) & (clients['income_type'] == 'компаньон'), 'total_income'] = companion_data_income_median
clients.loc[(clients['total_income'].isnull()) & (clients['income_type'] == 'пенсионер'), 'total_income'] = retiree_data_income_median
clients.loc[(clients['total_income'].isnull()) & (clients['income_type'] == 'госслужащий'), 'total_income'] = civil_servant_data_income_median
clients.loc[(clients['total_income'].isnull()) & (clients['income_type'] == 'предприниматель'), 'total_income'] = entrepreneur_data_income_median
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Аналогичным образом с помощью медианы заменим пропущенные значения в столбце с общим стажем работы days_employed. Определим максимальный и минимальный возраст клиентов.

In [7]:
print('Максимальный возраст клиента:', clients['dob_years'].max())
print('Минимальный возраст клиента:', clients['dob_years'].min())

Максимальный возраст клиента: 75
Минимальный возраст клиента: 0


Обнаружена очередная недостающая информация в данных. Для клиентов, возраст которых неизвестен, в столбце стоит значение 0 (ноль). Посчитаем количество таких клиентов.

In [8]:
print(clients.loc[clients['dob_years'] == 0, 'dob_years'].count())

101


У 101 клиента отсутствует информация о возрасте. Посмотрим, какой тип деятельности указан у клиентов с неизвестным возрастом.

In [9]:
print(clients.loc[clients['dob_years'] == 0, 'income_type'].value_counts())

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64


Используя медиану, восстановим значения в столбце с возрастом клиентов.

In [10]:
employee_data_age_median = employee_data['dob_years'].median()
print('Медиана возраста сотрудника составляет:', employee_data_age_median)

companion_data_age_median = companion_data['dob_years'].median()
print('Медиана возраста компаньона составляет:', companion_data_age_median)

retiree_data_age_median = retiree_data['dob_years'].median()
print('Медиана возраста пенсионера составляет:', retiree_data_age_median)

civil_servant_data_age_median = civil_servant_data['dob_years'].median()
print('Медиана возраста госслужащего составляет:', civil_servant_data_age_median)

Медиана возраста сотрудника составляет: 39.0
Медиана возраста компаньона составляет: 39.0
Медиана возраста пенсионера составляет: 60.0
Медиана возраста госслужащего составляет: 40.0


Восстановим значения в столбце 'dob_years' и выполним проверку восстановленных значений

In [11]:
clients.loc[(clients['dob_years'] == 0) & (clients['income_type'] == 'сотрудник'), 'dob_years'] = employee_data_age_median
clients.loc[(clients['dob_years'] == 0) & (clients['income_type'] == 'компаньон'), 'dob_years'] = companion_data_age_median
clients.loc[(clients['dob_years'] == 0) & (clients['income_type'] == 'пенсионер'), 'dob_years'] = retiree_data_age_median
clients.loc[(clients['dob_years'] == 0) & (clients['income_type'] == 'госслужащий'), 'dob_years'] = civil_servant_data_age_median
print(clients.loc[clients['dob_years'] == 0, 'dob_years'].count())

0


Посмотрим на максимальный и минимальный возраст клиентов после восставноления значений:

In [12]:
print('Максимальный возраст клиента:', clients['dob_years'].max())
print('Минимальный возраст клиента:', clients['dob_years'].min())

Максимальный возраст клиента: 75.0
Минимальный возраст клиента: 19.0


Зная возраст всех клиентов, можно восстановить пропущенные значения в столбце с общим стажем работы 'days_employed', используя медиану. Для этого разделим клиентов по возрасту на 4 равные группы: 19-32 лет, 33-46 лет, 47-60 лет, 61-75 лет.

In [13]:
young_days_employed_median = clients.loc[(clients['dob_years'] >= 19) & (clients['dob_years'] <= 32), 'days_employed'].median()
print('Медиана общего стажа клиента в категории 19-32 лет составляет:', young_days_employed_median)

middle1_days_employed_median = clients.loc[(clients['dob_years'] >= 33) & (clients['dob_years'] <= 46), 'days_employed'].median()
print('Медиана общего стажа клиента в категории 33-46 лет составляет:', middle1_days_employed_median)

middle2_days_employed_median = clients.loc[(clients['dob_years'] >= 47) & (clients['dob_years'] <= 60), 'days_employed'].median()
print('Медиана общего стажа клиента в категории 47-60 лет составляет:', middle2_days_employed_median)

age_days_employed_median = clients.loc[(clients['dob_years'] >= 61) & (clients['dob_years'] <= 75), 'days_employed'].median()
print('Медиана общего стажа клиента в категории 61-75 лет составляет:', age_days_employed_median)

Медиана общего стажа клиента в категории 19-32 лет составляет: -1098.3083782951276
Медиана общего стажа клиента в категории 33-46 лет составляет: -1763.563905000825
Медиана общего стажа клиента в категории 47-60 лет составляет: -1102.4379959880403
Медиана общего стажа клиента в категории 61-75 лет составляет: 356191.1376668496


Восстановим пропущенные значения в столбце 'days_employed':

In [14]:
clients.loc[(clients['dob_years'] >= 19) & (clients['dob_years'] <= 32), 'days_employed'] = young_days_employed_median
clients.loc[(clients['dob_years'] >= 33) & (clients['dob_years'] <= 46), 'days_employed'] = middle1_days_employed_median
clients.loc[(clients['dob_years'] >= 47) & (clients['dob_years'] <= 60), 'days_employed'] = middle2_days_employed_median
clients.loc[(clients['dob_years'] >= 61) & (clients['dob_years'] <= 75), 'days_employed'] = age_days_employed_median

Убедимся, что в датафрейме не осталось пропущенных значений:

In [15]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод**

На первом этапе предобработки данных выполнена работа по обработке пропущенных значений. Были обнаружены следующие аномалии:
1. В двух столбцах трудовой стаж в днях (days_employed) и ежемесячный доход (total_income) пропущено по 2174 значения, что составляет 10% всей выборки. Это довольно высокое значение, поэтому данные строки были восстановлены, а не исключены. Вероятно, данные пропали в результате технического сбоя.
2. Для 101 клиента, в столбце с возрастом стоит значение 0 (ноль). Вероятно, данные о возрасте были неизвестны и поэтому заменены нулевыми значениями.
Для восстановления значений по столбцам использовался метод медианы, т.к. разница между наибольшим и наименьшим значениями была достаточно велика и примерение среднеарифметического метода могло сильно исказить данные выборки.

### Замена типа данных

После выполнения работы по восстановлению пропущенных значений и просмотру информации о датафрейме можно заметить, что в столбце с возрастом клиентов 'dob_years' тип данных int64 (целочисленный) изменился на float64 (вещественный). Должно быть это произошло в следствие замены нулевых данных по возрасту на медиану, которая как раз имела тип float64.
Заменим тип данных float64 в столбце 'dob_years' на int64:

In [16]:
clients['dob_years'] = clients['dob_years'].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Выполнена замена вещественного типа данных float64 на целочисленный int64 с использованием простого метода astype('int'). Также в pandas возможно преобразование строковых значений в числовые. Однако, в данной работе этот метод не использовался.

### Обработка дубликатов

Для начала попробуем проверить наши данные с помощью стандартного метода duplicated():

In [17]:
print(clients.duplicated().sum())

54


Удалим из даных обнаруженные дубликаты:

In [18]:
clients = clients.drop_duplicates().reset_index(drop = True)

In [19]:
print(clients.duplicated().sum())

0


Поищем также дубликаты методом, возвращающим уникальные значения с их частотой:

In [20]:
print(clients['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Приведем данные в столбце с информацией о образовании к нижнему регистру методом lower():

In [21]:
clients['education'] = clients['education'].str.lower()
print(clients['education'].value_counts())

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [22]:
print(clients['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


Приведем данные в столбце с информацией о семейном положении к нижнему регистру методом lower():

In [23]:
clients['family_status'] = clients['family_status'].str.lower()
print(clients['family_status'].value_counts())

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [24]:
print(clients['gender'].value_counts())

F      14189
M       7281
XNA        1
Name: gender, dtype: int64


В столбце с информацией о поле клиентов обнаружилась строка с отсутствующим значением. Посмотрим на эту строку:

In [25]:
print(clients[clients['gender'] == 'XNA'])

       children  days_employed  dob_years            education  education_id  \
10690         0   -1098.308378         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10690  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10690  203905.157261  покупка недвижимости  


Исключим данную строку из выборки:

In [26]:
clients = clients.drop(clients[clients['gender'] == 'XNA'].index)
print(clients['gender'].value_counts())

F    14189
M     7281
Name: gender, dtype: int64


In [27]:
print(clients.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -1763.563905         42               высшее             0   
1          1   -1763.563905         36              среднее             1   
2          0   -1763.563905         33              среднее             1   
3          3   -1098.308378         32              среднее             1   
4          0   -1102.437996         53              среднее             1   
5          0   -1098.308378         27               высшее             0   
6          0   -1763.563905         43               высшее             0   
7          0   -1102.437996         50              среднее             1   
8          2   -1763.563905         35               высшее             0   
9          0   -1763.563905         41              среднее             1   
10         2   -1763.563905         36               высшее             0   
11         0   -1763.563905         40              среднее             1   

In [28]:
print(clients['income_type'].value_counts())

сотрудник          11091
компаньон           5079
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [29]:
print(clients['purpose'].value_counts())

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [30]:
print(clients.duplicated().sum())

17


Удалим повторно обнаружееные нам дубликаты после приведения всех строк к нижнему регистру:

In [31]:
clients = clients.drop_duplicates().reset_index(drop = True)

И выполним проверку оставшихся дубликатов:

In [32]:
print(clients.duplicated().sum())

0


**Вывод**

На данном этапе работы по поиску дубликатов был использован как стандартный метод pandas duplicated(), так и метод с возвращением уникальных значений с частотой их повторений и приведение текста к нижнему регистру с последующим корректным распределением информации по группам.
В итоге были удалены 54 дубликата строк методом duplicated(), выполнен ручной поиск дубликатов с учетом регистра, удалены 17 дубликатов строк после приведения к нижнему регистру, а также обнаружены данные для лемматизации.

### Лемматизация

В предыдущем разделе, после возвращения уникальных значений методом value_counts() по столбцу с целью получения кредита 'purpose', можно заметить, что часть значений повторяются, но в разных формах. А это означает, что их можно обработать с использованием функции лемматизации.
Для проверки можно вручную разделить значения на 5 категорий для проверки: 'жилье', 'автомобиль', 'свадьба', 'образование', 'недвижимость'.

In [33]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

words = ['жилье', 'автомобиль', 'свадьба', 'образование', 'недвижимость']


def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in words:
        if word in lemma:
            lemma = word
    return lemma


clients['purpose'] = clients['purpose'].apply(lemmatize)
print(clients.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -1763.563905         42               высшее             0   
1          1   -1763.563905         36              среднее             1   
2          0   -1763.563905         33              среднее             1   
3          3   -1098.308378         32              среднее             1   
4          0   -1102.437996         53              среднее             1   
5          0   -1098.308378         27               высшее             0   
6          0   -1763.563905         43               высшее             0   
7          0   -1102.437996         50              среднее             1   
8          2   -1763.563905         35               высшее             0   
9          0   -1763.563905         41              среднее             1   
10         2   -1763.563905         36               высшее             0   
11         0   -1763.563905         40              среднее             1   

Проверим предположение:

In [34]:
print(clients['purpose'].value_counts())

недвижимость    6350
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64


Поскольку жилье также относится к недвижимости, то данные две категории можно объединить:

In [35]:
clients.loc[(clients['purpose'] == 'жилье'), 'purpose'] = 'недвижимость'
print(clients['purpose'].value_counts())

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64


**Вывод**

Лемматизация значительно упрощает работу с данными и позволяет найти дублирование информации, которое невозможно выполнить стандартными методами pandas. В итоге удалось сократить информацию в столбце с различными целями кредита 'purpose' с 38 категорий всего до 4! Это позволит значительно упростить подготовку отчетной информации для Заказчика. 

### Категоризация данных

In [36]:
print(clients.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -1763.563905         42               высшее             0   
1          1   -1763.563905         36              среднее             1   
2          0   -1763.563905         33              среднее             1   
3          3   -1098.308378         32              среднее             1   
4          0   -1102.437996         53              среднее             1   
5          0   -1098.308378         27               высшее             0   
6          0   -1763.563905         43               высшее             0   
7          0   -1102.437996         50              среднее             1   
8          2   -1763.563905         35               высшее             0   
9          0   -1763.563905         41              среднее             1   
10         2   -1763.563905         36               высшее             0   
11         0   -1763.563905         40              среднее             1   

В данном виде таблицей для решения поставленных задач пользоваться неудобно, поэтому необходимо выполнить категоризацию данных.
Также отметим, что информация по образованию, семейному положению, полу и по цели получения кредита (на этапе лемматизации) уже категоризирована. Для выполнеения поставленной задачи требуется категоризировать данные по ежемесячной заработной плате и по возрасту. Для информации по количеству детей категорирование не требуется.

Уточним максимальный и минимальный ежемесячный доход клиентов, а также общее количество клиентов.

In [37]:
print('Максимальный ежемесячный доход клиента:', clients['total_income'].max())
print('Минимальный ежемесячный доход клиента:', clients['total_income'].min())
print('Медиана ежемесячного дохода клиента:', clients['total_income'].median())
print('Общее количество рассматриваемых клиентов:', clients['total_income'].count())

Максимальный ежемесячный доход клиента: 2265604.028722744
Минимальный ежемесячный доход клиента: 20667.26379327158
Медиана ежемесячного дохода клиента: 142594.39684740017
Общее количество рассматриваемых клиентов: 21453


In [38]:
def income_group(total_income):
    if total_income < 50000:
        return 'до 50000'
    if 50000 <= total_income < 100000:
        return 'от 50000 до 100000'
    if 100000 <= total_income < 150000:
        return 'от 100000 до 150000'
    if 150000 <= total_income < 200000:
        return 'от 150000 до 200000'
    if 200000 <= total_income < 250000:
        return 'от 200000 до 250000'
    if 250000 <= total_income < 500000:
        return 'от 250000 до 500000'
    return 'свыше 500000'

clients['income_group'] = clients['total_income'].apply(income_group)
print(clients['income_group'].value_counts())

от 100000 до 150000    7160
от 150000 до 200000    4764
от 50000 до 100000     4091
от 250000 до 500000    2591
от 200000 до 250000    2253
до 50000                372
свыше 500000            222
Name: income_group, dtype: int64


Категоризируем клиентов по возрасту: молодежь - до 35 лет; средний возраст - до 60 лет, пенсионеры - свыше 60 лет.

In [39]:
def age_group(dob_years):
    if dob_years <= 35:
        return 'молодежь'
    if dob_years < 60:
        return 'средний возраст'
    return 'пенсионеры'

clients['age_group'] = clients['dob_years'].apply(age_group)
print(clients['age_group'].value_counts())

средний возраст    12351
молодежь            6582
пенсионеры          2520
Name: age_group, dtype: int64


Упорядочим столбцы таблицы после категорирования:

In [40]:
data = clients
columns = ['children', 'days_employed', 'dob_years', 'age_group', 'education', 
          'education_id', 'family_status', 'family_status_id', 'gender',
         'income_type', 'debt', 'total_income', 'income_group', 'purpose']
clients = pd.DataFrame(data = data, columns = columns)

print(clients.head(5))

   children  days_employed  dob_years        age_group education  \
0         1   -1763.563905         42  средний возраст    высшее   
1         1   -1763.563905         36  средний возраст   среднее   
2         0   -1763.563905         33         молодежь   среднее   
3         3   -1098.308378         32         молодежь   среднее   
4         0   -1102.437996         53  средний возраст   среднее   

   education_id     family_status  family_status_id gender income_type  debt  \
0             0   женат / замужем                 0      F   сотрудник     0   
1             1   женат / замужем                 0      F   сотрудник     0   
2             1   женат / замужем                 0      M   сотрудник     0   
3             1   женат / замужем                 0      M   сотрудник     0   
4             1  гражданский брак                 1      F   пенсионер     0   

    total_income         income_group       purpose  
0  253875.639453  от 250000 до 500000  недвижимость  
1 

**Вывод**

Категоризировать данные можно двумя способами - по типу, и по группам. Категоризация данных позволяет преобразовать громоздкую таблицу и трудночитаемую информацию к простому и понятному виду. Поэтому категоризация является одним из заключительных этапов работы по предобработке данных.
На данном этапе была использована категоризация данных по группам в зависимости от возраста клиентов и от размера ежемесячного дохода клиента. В случае, если бы в полученных нами исходных данных было распределение на категории и подкатегории, то можно было бы воспользоваться категоризацией по типу.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Определим суммарное количество задолженностей по невозврату кредитов в зависимости от наличия детей и сформируем сводную таблицу методом data_pivot:

In [41]:
data_pivot = clients.pivot_table(index = ['children'], values = 'debt', aggfunc = 'sum')
print(data_pivot)

          debt
children      
-1           1
 0        1063
 1         444
 2         194
 3          27
 4           4
 5           0
 20          8


Из сводной таблицы видно, что в данные закрались ошибки - отрицательного количества детей быть не может. Количество детей 20 также свидетельствует об ошибочном занесении данных в таблицу. Внесем необходимые корректировки и повторно сформируем сводную таблицу. Дополнительно добавим столбцы с количеством детей у клиентов и рассчитаем долю невозвратов кредита:

In [42]:
clients.loc[(clients['children'] == -1), 'children'] = 1
clients.loc[(clients['children'] == 20), 'children'] = 2

data_pivot = clients.pivot_table(index = ['children'], values = 'debt', aggfunc = 'sum')
data_pivot['child_count'] = clients.groupby('children')['children'].count()
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['child_count']
print(data_pivot.sort_values('ratio', ascending = False))

          debt  child_count     ratio
children                             
4            4           41  0.097561
2          202         2128  0.094925
1          445         4855  0.091658
3           27          330  0.081818
0         1063        14090  0.075444
5            0            9  0.000000


**Вывод**

Из полученного результата можно сделать вывод, что на возврат кредита в срок влияет сам факт наличия детей, но не их количество. Самая низкая доля невозвратов кредита среди клиентов, не имеющих детей. Доля невозвратов среди людей, имеющих от 1 до 4 детей примерно одинакова и говорит об отсутствии зависимости между невозвратом кредита и количеством детей. Можно предположить, что в неблагоприятной финансовой ситуации клиенты предпочитают обеспечивать всем необходимым своих детей, чем возвращать долг банку.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сформируем сводную таблицу методом df.groupby().agg(). В таблице будет рассмотрена зависимость между количеством клиентов с различным семейным статусом и клиентов, срок возврата кредита по которым истек:

In [43]:
family_status_group = clients.groupby('family_status').agg({'family_status': ['count'], 'debt': ['sum']})
family_status_group['ratio'] = family_status_group['debt']['sum'] / family_status_group['family_status']['count']
print(family_status_group.sort_values('ratio', ascending = False))

                      family_status debt     ratio
                              count  sum          
family_status                                     
не женат / не замужем          2810  274  0.097509
гражданский брак               4150  388  0.093494
женат / замужем               12339  931  0.075452
в разводе                      1195   85  0.071130
вдовец / вдова                  959   63  0.065693


**Вывод**

Полученные результаты свидетельствуют о наличии зависимости между семейным положением и возвратом кредита в срок. Условно можно разделить полученные результаты на 2 группы, учитывая близость их значений. Первая группа - с наименьшим риском невозврата кредита - это люди, которые состоят в браке в настоящий момент, либо состояли в браке ранее. Это говорит о том, что брак учит людей ответственности во всех жизненных сферах. Вторая граппа - с наибольшим риском невозврата кредита - это люди, которые не состоят в браке, либо состоят, но в гражданском. Они хотят жить свободной жизнью, не обременяя себя обязательствами перед другими людьми, а также и банком, и могут спокойно отнестись к задолженности по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сформируем сводную таблицу методом df.groupby().agg(). В таблице будет рассмотрена зависимость между количеством клиентов, сгруппированных в зависимости от размера ежемесячного дохода и количеством кредитов, срок возврата по которым истек:

In [44]:
income_debt_group = clients.groupby('income_group').agg({'income_group': ['count'], 'debt': ['sum']})
income_debt_group['ratio'] = income_debt_group['debt']['sum'] / income_debt_group['income_group']['count']
print(income_debt_group.sort_values('ratio', ascending = False))

                    income_group debt     ratio
                           count  sum          
income_group                                   
от 100000 до 150000         7160  624  0.087151
от 150000 до 200000         4764  405  0.085013
от 50000 до 100000          4091  331  0.080909
от 200000 до 250000         2253  164  0.072792
от 250000 до 500000         2591  180  0.069471
свыше 500000                 222   14  0.063063
до 50000                     372   23  0.061828


**Вывод**

Полученные результаты свидетельствуют о наличии зависимости между уровнем дохода и возвратом кредита в срок. Однако, итоги получились достаточно неоднозначными, которые было бы очень непросто предсказать заранее. Можно было бы предположить, что у людей с наименьшим доходом риск невозврата кредита будет высоким. Но на практике оказалось, что наименьший риск невозврата кредита как раз либо у наименее обеспеченных клиентов, либо у высокообеспеченных. А наиболее высокий риск невозврата кредита банку у подавляющего большинства клиентов, имеющий средний уровень дохода. 
Это можно объяснить тем, что люди, имеющие низкий доход умеют правильно им распоряжаться и не берут на себя чрезмерные долговые обязательства. Люди с высоким доходом, соответственно, имеют больше возможностей для погашения кредита в срок. В отлиечие от них, люди со средним достатком не могут правильно распоряжаться своими финансами и склонны брать на себя чрезмерную долговую нагрузку, ошибочно полагая, что смогут безболезненно с ней справиться.

- Как разные цели кредита влияют на его возврат в срок?

Сформируем сводную таблицу методом df.groupby().agg(). В таблице будет рассмотрена зависимость между количеством клиентов, сгруппированных в зависимости от цели получения кредита и количеством кредитов, срок возврата по которым истек:

In [45]:
purpose_debt_group = clients.groupby('purpose').agg({'purpose': ['count'], 'debt': ['sum']})
purpose_debt_group['ratio'] = purpose_debt_group['debt']['sum'] / purpose_debt_group['purpose']['count']
print(purpose_debt_group.sort_values('ratio', ascending = False))

             purpose debt     ratio
               count  sum          
purpose                            
автомобиль      4306  403  0.093590
образование     4013  370  0.092200
свадьба         2324  186  0.080034
недвижимость   10810  782  0.072340


**Вывод**

Из полученных данных следует вывод о значительном влиянии цели кредита на его возврат. Наименьший риск невозврата кредита по целям, связанным с образованием и операциям с недвижимостью. Наиболее высокий риск невозврата - по целям, связанным с приобретением автомобиля и образованием.
Дело в том, что приобретение недвижимости и создание семьи весьма значимы для человека в долгосрочной перспективе и говорят о серьезности его намерений.
Образование также важно для человека, но в отличие от вышеуказанных целей, не гарантирует наличия дохода в краткосрочной перспективе. Кроме того, сам факт получения образования не для всех людей имеет первостепенное значение. И тем более приобретение автомобиля в современном мире для многих скорее является предметом роскоши, статуса и комфорта, чем предметом первой необходимости. 

## Шаг 4. Общий вывод

В данной проектной работе, выполненной для Заказчика — кредитного отдела банка, выполнен анализ влияния семейного положения, количества детей клиента, уровня ежемесячного дохода, а также целей кредита на факт погашения кредита в срок.
Для этого были выполнены следующие этапы в части предобработки данных:
1. Определины и заполнены пропущенные значения;
2. Исключено дублирование информации;
3. Выполнено лемматизирование в значениях столбца с целями получения кредита;
4. Выполнена категоризация данных;
5. Даны ответы на вопросы, поставленные Заказчиком.
Полученные результаты не всегда было возможно однозначно предсказать заранее.
Подводя общий итог: 
наиболее неблагоприятные клиенты для банка с точки зрения возможности невозврата кредита в срок - клиенты, имеющие детей, не состоящие в браке (в т.ч. в гражданском), имеющие средний ежемесячный доход и указывающие в целях на получение кредита приобретение автомобиля, либо получение образования.
Соответственно, клиенты с наименьшим риском невозврата кредита в срок - клиенты, не имеющие детей, состоящие, либо состоявшие ранее в браке, имеющие низкий, либо наоборот высокий ежемесячный доход и получающие кредит на операции с недвижимостью, либо свадьбу.